# Chapter 4: Transforming vectors and graphics

**NOTE** For most of the examples in this chapter, the code is in other .py files in this directory, rather than directly in this Notebook.  You can run the other files in the notebook below, and a PyGame window will open up.  For instance `!python script.py` runs `script.py` as if you're running it from the command line. When you're done admiring the graphics the scripts create, you can close the window and the Jupyter cell will finish computing.

Make sure to follow the instructions from Appendix C before beginning this chapter, including

`pip install PyGame`

`pip install PyOpenGL`

# 4.1 Transforming 3D objects

## 4.1.1 Drawing a transformed object

symbol **`!`** tells jypiter notebook to execute scrip in command lie 

In [1]:
!python draw_teapot.py

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
!python scale_teapot.py

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
!python scale_translate_teapot.py

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


## 4.1.2 Composing vector transformations

When function end with `()` -- `parentheses`, it means that function will be executed  
* [Programming Terms: First-Class Functions](https://www.youtube.com/watch?v=kr0mpwqttM0)  
* [Currying](https://www.python-course.eu/currying_in_python.php#General-Idea)

In [53]:
def compose(f1,f2):
    def new_function(input):
        return f1(f2(input))
    return new_function

In [54]:
def square(x):
    return x**2
def adding(x):
    return x+x

In [55]:
square_adding = compose(square, adding)
square_adding(1)

4

In [56]:
from vectors import scale 

def scale2(v): 
    return scale(2.0, v)

In [57]:
def polygon_map(transformation, polygons):
    return [
        [transformation(vertex) for vertex in triangle]
        for triangle in polygons
    ]

In [58]:
from teapot import load_triangles

polygon_map(scale2, load_triangles())[0]

[(1.4, 1.0521480000000003, 0.0),
 (1.2734839999999998, 1.124108, -0.541836),
 (1.2915, 1.0521480000000003, -0.5495)]

In [59]:
def scale_by(scalar):
    def new_function(v):
        return scale(scalar, v)
    return new_function

In [60]:
scale2 = scale_by(2)
scale2((1,1,1))

(2, 2, 2)

In [61]:
scale_by(2)((1,1,1))

(2, 2, 2)

In [62]:
from vectors import add

def translate_by(xform):
    def new_function(v):
        return add((xform), (v))
    return new_function

In [63]:
translate2 = translate_by((0,5,0))
translate2((1,1,1))

(1, 6, 1)

In [64]:
translate_by((0,5,0))((1,1,1))

(1, 6, 1)

## 4.1.3 Rotating an object about an axis

In [65]:
!python rotate_teapot.py

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [71]:
from vectors import to_cartesian, to_polar

def rotate2d(angle, vector):
    l,a = to_polar(vector)
    return to_cartesian((l, a+angle))

In [75]:
def rotate_z(angle, vector): 
      x,y,z = vector 
      new_x, new_y = rotate2d(angle, (x,y)) 
      return new_x, new_y, z

In [78]:
rotate_z(.7, (1,1,1))

(0.12062450004679749, 1.4090598745221794, 1)

In [79]:
def rotate_by_z(angle):
    def new_function(v):
        return rotate_z(angle, v)
    return new_function

In [88]:
from draw_model import draw_model
from math import pi

draw_model(polygon_map(rotate_by_z(pi/4.), load_triangles()))

NameError: name 'quit' is not defined

In [85]:
def rotate_x(angle, vector): 
      x,y,z = vector 
      new_y, new_z = rotate2d(angle, (y,z)) 
      return x, new_y, new_z 
  
def rotate_x_by(angle): 
    def new_function(v): 
        return rotate_x(angle,v) 
    return new_function

In [87]:
draw_model(polygon_map(rotate_x_by(pi/2.), load_triangles()))

NameError: name 'quit' is not defined

## What is the purpose of currying?
## Go back and revisit chapter

When function end with `()` -- `parentheses`, it means that function will be executed  
* [Programming Terms: First-Class Functions](https://www.youtube.com/watch?v=kr0mpwqttM0)  
* [Currying](https://www.python-course.eu/currying_in_python.php#General-Idea)